### 10/26時点のrecover-original-textのイケ具合を確認しつつ、テキストの抜粋基準について検討する
* まずイケてないです
* 「ああああああ」「ﾌﾞﾘｭﾌﾞﾘｭ」「ええんやで」「クレメンス」的なミームが入ってるやつはたくさんヒットしてしまう
    * とりあえず意味変わってなさそうやしいいんじゃね？
* 顔文字のバリエーションが多すぎて突破困難
    * とりあえずjuman通るようにreplaceして、他は無視

In [1]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
train_df = pd.read_feather("input/dataset_with_original_text/train_with_original_text.feather")
test_df = pd.read_feather("input/dataset_with_original_text/test_with_original_text.feather")

In [3]:
from bert_utils import *
train_df["clean_text"] = train_df["original_text"].map(lambda x: original_text_preprocess(x))

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

****** SEED fixed : 256 ******




In [4]:
tk1 = define_tokenizer("cl-tohoku/bert-base-japanese-whole-word-masking")
tk2 = define_tokenizer("nlp-waseda/roberta-large-japanese-seq512")

In [5]:
# ~3sec --
train_df["clean_text1"] = train_df["original_text"].parallel_map(lambda x: original_text_preprocess(x, use_juman=False))

In [6]:
train_df["clean_text2"] = train_df["original_text"].parallel_map(lambda x: original_text_preprocess(x, use_juman=True))

### トークン数に関する検討 --

### 非Juman系 ]333 --

In [7]:
train_df["clean_text1"].map(lambda x: len(tk1.tokenize(x))).sort_values()

506       7
2615     12
1976     15
1265     15
2661     17
       ... 
2811    271
3297    275
4400    284
2853    319
4989    333
Name: clean_text1, Length: 5256, dtype: int64

### Juman系 :346 --

In [8]:
train_df["clean_text2"].map(lambda x: len(tk2.tokenize(x))).sort_values()

506       8
2615     13
1265     14
2902     16
1976     18
       ... 
4400    267
4067    279
3297    284
2853    315
4989    346
Name: clean_text2, Length: 5256, dtype: int64

In [31]:
tk1.encode_plus(
    train_df["clean_text1"][0],
    add_special_tokens=True,
    padding="max_length",
    max_length=2)

{'input_ids': [2, 23135, 7, 21117, 5, 2434, 28836, 15, 16, 4831, 80, 1058, 75, 11218, 14209, 9968, 29360, 20856, 3635, 2889, 20, 3318, 708, 14143, 28563, 28488, 34, 3635, 3, 23135, 7, 1879, 26, 20, 16, 18, 21, 28514, 13, 5, 633, 15060, 4799, 15872, 7, 34, 120, 29, 2935, 3, 1221, 28516, 28482, 20663, 28457, 1058, 75, 54, 1091, 12, 9, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
# for i in range(4800, train_df.shape[0]):
#     print(i)
#     t = original_text_preprocess(train_df.loc[i, "original_text"], use_juman=True)

#### juman_parseで引っかかったやつ

In [10]:
train_df.loc[3218, "clean_text"]

'ネトウヨってバカだなぁ韓国側が反日運動する時は散叩くクセに自分たちはヘイトスピーチ禁止されて怒り狂ってるんだwwwお前らの言ってる事って結局のところ反日やってる韓国人と全く変わんないって事に気づいた方がいいよ [SEP]日本人に対するヘイトは含まないって言ってるのも理解できないとは本当に日本人じゃないんだろうな[SEP]じゃあお前日本人に対するヘイトスピーチ見た事あんのかよ俺は今まで韓国や中国に対するものしか見た事ねーぞ[SEP]さすが地震をお祝いして国旗を平気に燃やす(日本では犯罪)民族は言うことが違うな'

In [11]:
tt = train_df.loc[3239, "clean_text"]
tt

'志位るず界隈しばき隊界隈はこれからパヨクっていわれちゃうの?wwwwww[SEP]>パヨクそれいただきましたw[SEP]相変わらずこういうときの真紅ってキレッキレやと思うねんwwww'

In [12]:
tt = train_df.loc[4102, "original_text"]
tt

'おやおや、移民難民は受け入れろだの救えだのおっしゃる上級国民様はダンマリかいｗ\tその上級国民って誰？\t○ニクロのしゃっちょさんとかアカヒなどのマスゴミ様とか\tそっか __BR__ ありがとう<(_ _)>\n'

In [13]:
tt = train_df.loc[4871, "original_text"]
tt

'bcxx氏のアカが消えていたのですが、何かあったんですかね？(名の有るパヨクの何人かも11/14～11/15以降はパヨった書き込み無しで、リツイートぐらい、確認出来た所だと、パヨって暴れているのは在日韓国人の人達ばかり)\t竹内のツイアカは消えてないぞ __BR__ いくつかの書き込みを消しただけじゃないかな\tそうなんですね __BR__ もう一度探してみます __BR__ 有難うございました __BR__ m(_ _)m\n'

### 顔文字チェックはtestでもやっておく --

In [14]:
# for i in range(0, test_df.shape[0]):
#     print(i)
#     t = original_text_preprocess(test_df.loc[i, "original_text"], use_juman=True)

### 前処理が思った動きしてなくない？ --

* 3点リーダが消えない

In [15]:
train_df.loc[180, "original_text"].replace("…", "")

'5MRはどうしてわたしの指摘スルーしたの？ｗ __BR__ なのにどうしてガンガン自分の意見だしてるの？ｗ __BR__ どの面さげてｗ\tお前に興味ないからかな\tはいにげたっとｗ __BR__ 一発でつぶれてたのかｗ __BR__ あれだけ勇んでたのにｗｗｗ __BR__ スルーした君はひとに意見求める資格なーっしｗ\n'

In [16]:
train_df.loc[:, ["label", "clean_text"]]

,label,clean_text
0,0,まともにLINEの返信してくれないんだけど諦めなきゃダメだろフラれたらギクシャクするだろ[S...
1,0,柳なんでこんないい時と悪い時の波があるんや[SEP]最近はアヘアヘQSマンやない? イイ!(...
2,1,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ甘えるな[...
3,0,よくよく思えば川上は配布にしたらとんでもなく有能だよなガチャから引いたら圧倒的歓喜レベルやで...
4,0,押井は原作レイプの専門家だから原作マンガの真意を誤解させることに関してはプロだがそれ以外には...
...,...,...
5251,0,車じゃなくてもよくない?ケーブル網を張り巡らせてリフトみたいなのを付けるとか[SEP]それな...
5252,0,今からでも野間か松山を出せばいいのに最近のエルは正直いって期待薄[SEP]左やぞ?出すなら下...
5253,0,日本によってけんけんガクガクの議論を持たらされた韓国は被害者日本人がしっかり考えないと10億...
5254,0,安楽死を合法にしたら若者殺到するんだろうなあ[SEP]ゴムボート買って沖まで漕いで行ったら?...


#### 繰り返し文字をどう抜いていくか
* 単に繰り返し指定だと「ええんやで」が「えんやで」になる

In [17]:
t = train_df.loc[210, "original_text"]
t

"(´・ω・｀)ドキッ\t(o'ω'n)「あああああああああああああああああああああああああああああああ！！！！！！！！！！！（ﾌﾞﾘﾌﾞﾘﾌﾞﾘﾌﾞﾘｭﾘｭﾘｭﾘｭﾘｭﾘｭ！！！！！！ﾌﾞﾂﾁﾁﾌﾞﾌﾞﾌﾞﾁﾁﾁﾁﾌﾞﾘﾘｲﾘﾌﾞﾌﾞﾌﾞﾌﾞｩｩｩｩｯｯｯ！！！！！！！）」\n"

In [18]:
list(nchars(t, 2))

['あああああああああああああああああああああああああああああああ',
 '！！！！！！！！！！！',
 '！！！！！！',
 'ﾁﾁ',
 'ﾁﾁﾁﾁ',
 'ﾘﾘ',
 'ｩｩｩｩ',
 'ｯｯｯ',
 '！！！！！！！']

### 全件print(train) --

In [19]:
# pd.set_option("display.max_rows", 20000)
# pd.set_option("display.max_colwidth", None)
# train_df.loc[train_df.isnull().sum(axis=1).sort_values().index, ["label", "text", "original_text"]]